# Example usage of `PFNs4BOSampler`

This notebook demonstrates a simple usage of `PFNs4BOSampler` in [OptunaHub](https://hub.optuna.org/).
This sampler uses Prior-data Fitted Networks (PFNs) as a surrogate model for Bayesian optimization.
See the [paper](https://arxiv.org/abs/2305.17535) for more details.

- Samuel Müller, Matthias Feurer, Noah Hollmann, and Frank Hutter. 2023. PFNs4BO: in-context learning for Bayesian optimization. In Proceedings of the 40th International Conference on Machine Learning (ICML'23), Vol. 202. JMLR.org, Article 1056, 25444–25470.

## Step 1: Installation

The dependencies of this example are listed in [`requirements.txt`](./requirements.txt). Please install them as follows:

In [ ]:
%pip install -q --progress-bar off -r requirements.txt

## Step 2: Download model checkpoint

The [official repository of PFNs4BO](https://github.com/automl/PFNs4BO) provides pre-trained models of PFNs. Please download one as follows:

In [ ]:
!curl -LO https://github.com/automl/PFNs4BO/raw/main/pfns4bo/final_models/hebo_morebudget_9_unused_features_3_userpriorperdim2_8.pt.gz
!gunzip hebo_morebudget_9_unused_features_3_userpriorperdim2_8.pt.gz

## Step 3: Define objective function

As a demonstration, we use a simple quadratic function as an objective function.

In [ ]:
import optuna
import optunahub
import torch
import warnings

def objective(trial: optuna.Trial) -> float:
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2

## Step 4: Load `PFNs4BOSampler`

With `optunahub.load_module`, you can use modules in [OptunaHub](https://hub.optuna.org/) in your code.
In this case, a module defined in [samplers/pfns4bo](https://hub.optuna.org/samplers/pfns4bo/) is loaded, and you can instantiate `PFNs4BOSampler` in it.

In this example, we set the previously downloaded model to `PFNs4BOSampler`.

In [ ]:
module = optunahub.load_module("samplers/pfns4bo")

sampler = module.PFNs4BOSampler(
    prior=torch.load("hebo_morebudget_9_unused_features_3_userpriorperdim2_8.pt"),
)

## Step 5: Run optimization

You can optimize the objective function with `PFNs4BOSampler` as usual.

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=100)

In [ ]:
optuna.visualization.plot_optimization_history(study)